In [1]:
from numpy_artm import *

In [2]:
train_n_dw_matrix, test_n_dw_matrix, token_2_num, num_2_token, doc_occurences, doc_cooccurences = prepare_nips_dataset('/home/tylorn/NIPS_1987-2015.csv', calc_cooccurences=True, train_test_split=0.8)

Read file lines: 0
Read file lines: 1000
Read file lines: 2000
Read file lines: 3000
Read file lines: 4000
Read file lines: 5000
Read file lines: 6000
Read file lines: 7000
Read file lines: 8000
Read file lines: 9000
Read file lines: 10000
Read file lines: 11000
Processed documents: 0
Processed documents: 100
Processed documents: 200
Processed documents: 300
Processed documents: 400
Processed documents: 500
Processed documents: 600
Processed documents: 700
Processed documents: 800
Processed documents: 900
Processed documents: 1000
Processed documents: 1100
Processed documents: 1200
Processed documents: 1300
Processed documents: 1400
Processed documents: 1500
Processed documents: 1600
Processed documents: 1700
Processed documents: 1800
Processed documents: 1900
Processed documents: 2000
Processed documents: 2100
Processed documents: 2200
Processed documents: 2300
Processed documents: 2400
Processed documents: 2500
Processed documents: 2600
Processed documents: 2700
Processed documents: 

In [18]:
with open('thetaless_exp/nips_doc_occurences', 'w') as f:
    f.write('{}\n'.format(train_n_dw_matrix.shape[0]))
    for word_num, value in doc_occurences.iteritems():
        f.write('{},{}\n'.format(word_num, value))

In [17]:
with open('thetaless_exp/nips_doc_cooccurences', 'w') as f:
    f.write('{}\n'.format(train_n_dw_matrix.shape[0]))
    for (fst_word_num, snd_word_num), value in doc_cooccurences.iteritems():
        f.write('{},{},{}\n'.format(fst_word_num, snd_word_num, value))

In [19]:
D, W = train_n_dw_matrix.shape
T = 30

np.random.seed(42)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64) + 0.1
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = create_reg_lda(0., 0.)

_train_perplexity = artm_calc_perplexity_factory(train_n_dw_matrix) 
_test_perplexity = artm_calc_perplexity_factory(test_n_dw_matrix)
_top5_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 5)
_top10_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 10)
_top20_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 20)

def callback(it, phi, theta):
    if it % 10 == 0:
        print 'Iteration', it
        print '\tTrain perplexity:   {}'.format(_train_perplexity(phi, theta))
        print '\tTest perplexity:    {}'.format(_test_perplexity(phi, theta))
        print '\tTopics correlation  {}'.format(artm_calc_topic_correlation(phi))
        print '\tPhi sparsity        {}'.format(1. * np.sum(phi < 1e-20) / np.sum(phi >= 0))
        print '\tTheta sparsity      {}'.format(1. * np.sum(theta < 0.01) / np.sum(theta >= 0))
        print '\tTop5 PMI            {}'.format(_top5_pmi(phi))
        print '\tTop10 PMI           {}'.format(_top10_pmi(phi))
        print '\tTop20 PMI           {}'.format(_top20_pmi(phi))
        print '\tAvg kernel size     {}'.format(np.mean(artm_get_kernels_sizes(phi)))
        print '\tAvg kernel jacards  {}'.format(artm_get_avg_pairwise_kernels_jacards(phi))


phi, theta = em_optimization(
    n_dw_matrix=train_n_dw_matrix, 
    phi_matrix=phi_matrix,
    theta_matrix=theta_matrix,
    regularization_list=regularization_list,
    iters_count=101,
    loss_function=LogFunction(),
    iteration_callback=callback
)

Iteration 0
	Train perplexity:   2720.04221623
	Test perplexity:    2724.12276922
	Topics correlation  0.000959412810628
	Phi sparsity        0.0
	Theta sparsity      5.7431656329e-06
	Top5 PMI            0.00843759826024
	Top10 PMI           0.00964322052527
	Top20 PMI           0.0116327800629
	Avg kernel size     1962.1
	Avg kernel jacards  0.643754002782
Iteration 10
	Train perplexity:   2468.1398764
	Test perplexity:    2481.24200012
	Topics correlation  0.000951899621516
	Phi sparsity        0.0
	Theta sparsity      0.0500344589938
	Top5 PMI            0.0429180561786
	Top10 PMI           0.0335836363043
	Top20 PMI           0.0443712010506
	Avg kernel size     1899.2
	Avg kernel jacards  0.521716475007
Iteration 20
	Train perplexity:   1886.37424435
	Test perplexity:    1923.57277845
	Topics correlation  0.000910463706454
	Phi sparsity        0.00920934019599
	Theta sparsity      0.544136227889
	Top5 PMI            0.15051137611
	Top10 PMI           0.140200309252
	Top20 PMI    

In [20]:
D, W = train_n_dw_matrix.shape
T = 30

np.random.seed(42)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64) + 0.1
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = create_reg_lda(0., 0.)

_train_perplexity = artm_calc_perplexity_factory(train_n_dw_matrix) 
_test_perplexity = artm_calc_perplexity_factory(test_n_dw_matrix)
_top5_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 5)
_top10_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 10)
_top20_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 20)

def callback(it, phi, theta):
    if it % 10 == 0:
        print 'Iteration', it
        print '\tTrain perplexity:   {}'.format(_train_perplexity(phi, theta))
        print '\tTest perplexity:    {}'.format(_test_perplexity(phi, theta))
        print '\tTopics correlation  {}'.format(artm_calc_topic_correlation(phi))
        print '\tPhi sparsity        {}'.format(1. * np.sum(phi < 1e-20) / np.sum(phi >= 0))
        print '\tTheta sparsity      {}'.format(1. * np.sum(theta < 0.01) / np.sum(theta >= 0))
        print '\tTop5 PMI            {}'.format(_top5_pmi(phi))
        print '\tTop10 PMI           {}'.format(_top10_pmi(phi))
        print '\tTop20 PMI           {}'.format(_top20_pmi(phi))
        print '\tAvg kernel size     {}'.format(np.mean(artm_get_kernels_sizes(phi)))
        print '\tAvg kernel jacards  {}'.format(artm_get_avg_pairwise_kernels_jacards(phi))


phi, theta = artm_thetaless_em_optimization(
    n_dw_matrix=train_n_dw_matrix, 
    phi_matrix=phi_matrix,
    regularization_list=regularization_list,
    iters_count=101,
    iteration_callback=callback
)

Iteration 0
	Train perplexity:   2719.73994176
	Test perplexity:    2723.83745613
	Topics correlation  0.000957284058693
	Phi sparsity        3.19869725784e-05
	Theta sparsity      5.7431656329e-06
	Top5 PMI            0.00870857804955
	Top10 PMI           0.0097819585326
	Top20 PMI           0.011836723299
	Avg kernel size     1959.7
	Avg kernel jacards  0.640570548258
Iteration 10
	Train perplexity:   2311.71825698
	Test perplexity:    2322.63642141
	Topics correlation  0.000904914553454
	Phi sparsity        0.238637354968
	Theta sparsity      0.0144785205605
	Top5 PMI            0.195146597156
	Top10 PMI           0.160346179544
	Top20 PMI           0.146534836899
	Avg kernel size     1612.0
	Avg kernel jacards  0.350004798184
Iteration 20
	Train perplexity:   2038.45190935
	Test perplexity:    2063.07347351
	Topics correlation  0.000519769675963
	Phi sparsity        0.426418331443
	Theta sparsity      0.130984378589
	Top5 PMI            0.221438791557
	Top10 PMI           0.2351375

In [21]:
D, W = train_n_dw_matrix.shape
T = 30

np.random.seed(42)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64) + 0.1
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = create_reg_lda(-0.1, 0.)

_train_perplexity = artm_calc_perplexity_factory(train_n_dw_matrix) 
_test_perplexity = artm_calc_perplexity_factory(test_n_dw_matrix)
_top5_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 5)
_top10_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 10)
_top20_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 20)

def callback(it, phi, theta):
    if it % 10 == 0:
        print 'Iteration', it
        print '\tTrain perplexity:   {}'.format(_train_perplexity(phi, theta))
        print '\tTest perplexity:    {}'.format(_test_perplexity(phi, theta))
        print '\tTopics correlation  {}'.format(artm_calc_topic_correlation(phi))
        print '\tPhi sparsity        {}'.format(1. * np.sum(phi < 1e-20) / np.sum(phi >= 0))
        print '\tTheta sparsity      {}'.format(1. * np.sum(theta < 0.01) / np.sum(theta >= 0))
        print '\tTop5 PMI            {}'.format(_top5_pmi(phi))
        print '\tTop10 PMI           {}'.format(_top10_pmi(phi))
        print '\tTop20 PMI           {}'.format(_top20_pmi(phi))
        print '\tAvg kernel size     {}'.format(np.mean(artm_get_kernels_sizes(phi)))
        print '\tAvg kernel jacards  {}'.format(artm_get_avg_pairwise_kernels_jacards(phi))


phi, theta = em_optimization(
    n_dw_matrix=train_n_dw_matrix, 
    phi_matrix=phi_matrix,
    theta_matrix=theta_matrix,
    regularization_list=regularization_list,
    iters_count=101,
    loss_function=LogFunction(),
    iteration_callback=callback
)

Iteration 0
	Train perplexity:   2720.15673254
	Test perplexity:    2724.36116087
	Topics correlation  0.000966241531634
	Phi sparsity        0.0
	Theta sparsity      5.7431656329e-06
	Top5 PMI            0.00843759826024
	Top10 PMI           0.00964322052527
	Top20 PMI           0.0116327800629
	Avg kernel size     1962.1
	Avg kernel jacards  0.643754002782
Iteration 10
	Train perplexity:   2462.42671959
	Test perplexity:    2476.19263244
	Topics correlation  0.000957931532781
	Phi sparsity        0.108627758876
	Theta sparsity      0.0527165173444
	Top5 PMI            0.0429180561786
	Top10 PMI           0.033600409043
	Top20 PMI           0.0444077265423
	Avg kernel size     1912.8
	Avg kernel jacards  0.510745074936
Iteration 20
	Train perplexity:   1891.95369114
	Test perplexity:    1929.89174713
	Topics correlation  0.000912156537334
	Phi sparsity        0.479193928291
	Theta sparsity      0.540408913393
	Top5 PMI            0.154891362902
	Top10 PMI           0.140998011851
	Top

In [22]:
D, W = train_n_dw_matrix.shape
T = 30

np.random.seed(42)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64) + 0.1
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = create_reg_lda(-0.1, 0.)

_train_perplexity = artm_calc_perplexity_factory(train_n_dw_matrix) 
_test_perplexity = artm_calc_perplexity_factory(test_n_dw_matrix)
_top5_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 5)
_top10_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 10)
_top20_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 20)

def callback(it, phi, theta):
    if it % 10 == 0:
        print 'Iteration', it
        print '\tTrain perplexity:   {}'.format(_train_perplexity(phi, theta))
        print '\tTest perplexity:    {}'.format(_test_perplexity(phi, theta))
        print '\tTopics correlation  {}'.format(artm_calc_topic_correlation(phi))
        print '\tPhi sparsity        {}'.format(1. * np.sum(phi < 1e-20) / np.sum(phi >= 0))
        print '\tTheta sparsity      {}'.format(1. * np.sum(theta < 0.01) / np.sum(theta >= 0))
        print '\tTop5 PMI            {}'.format(_top5_pmi(phi))
        print '\tTop10 PMI           {}'.format(_top10_pmi(phi))
        print '\tTop20 PMI           {}'.format(_top20_pmi(phi))
        print '\tAvg kernel size     {}'.format(np.mean(artm_get_kernels_sizes(phi)))
        print '\tAvg kernel jacards  {}'.format(artm_get_avg_pairwise_kernels_jacards(phi))


phi, theta = artm_thetaless_em_optimization(
    n_dw_matrix=train_n_dw_matrix, 
    phi_matrix=phi_matrix,
    regularization_list=regularization_list,
    iters_count=101,
    iteration_callback=callback
)

Iteration 0
	Train perplexity:   2719.8456658
	Test perplexity:    2724.06746497
	Topics correlation  0.000964106457913
	Phi sparsity        3.19869725784e-05
	Theta sparsity      5.7431656329e-06
	Top5 PMI            0.00870857804955
	Top10 PMI           0.0097819585326
	Top20 PMI           0.011836723299
	Avg kernel size     1959.7
	Avg kernel jacards  0.640570548258
Iteration 10
	Train perplexity:   2306.40828041
	Test perplexity:    2317.65340451
	Topics correlation  0.000907112361536
	Phi sparsity        0.386987117974
	Theta sparsity      0.0146623018608
	Top5 PMI            0.197216795987
	Top10 PMI           0.160749901106
	Top20 PMI           0.150739016247
	Avg kernel size     1617.6
	Avg kernel jacards  0.340803563172
Iteration 20
	Train perplexity:   2039.0218657
	Test perplexity:    2064.13720614
	Topics correlation  0.000519149746023
	Phi sparsity        0.728875512519
	Theta sparsity      0.132081323225
	Top5 PMI            0.22056061006
	Top10 PMI           0.2348755230

In [23]:
D, W = train_n_dw_matrix.shape
T = 30

np.random.seed(42)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64) + 0.1
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = create_reg_lda(0.1, 0.)

_train_perplexity = artm_calc_perplexity_factory(train_n_dw_matrix) 
_test_perplexity = artm_calc_perplexity_factory(test_n_dw_matrix)
_top5_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 5)
_top10_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 10)
_top20_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 20)

def callback(it, phi, theta):
    if it % 10 == 0:
        print 'Iteration', it
        print '\tTrain perplexity:   {}'.format(_train_perplexity(phi, theta))
        print '\tTest perplexity:    {}'.format(_test_perplexity(phi, theta))
        print '\tTopics correlation  {}'.format(artm_calc_topic_correlation(phi))
        print '\tPhi sparsity        {}'.format(1. * np.sum(phi < 1e-20) / np.sum(phi >= 0))
        print '\tTheta sparsity      {}'.format(1. * np.sum(theta < 0.01) / np.sum(theta >= 0))
        print '\tTop5 PMI            {}'.format(_top5_pmi(phi))
        print '\tTop10 PMI           {}'.format(_top10_pmi(phi))
        print '\tTop20 PMI           {}'.format(_top20_pmi(phi))
        print '\tAvg kernel size     {}'.format(np.mean(artm_get_kernels_sizes(phi)))
        print '\tAvg kernel jacards  {}'.format(artm_get_avg_pairwise_kernels_jacards(phi))


phi, theta = em_optimization(
    n_dw_matrix=train_n_dw_matrix, 
    phi_matrix=phi_matrix,
    theta_matrix=theta_matrix,
    regularization_list=regularization_list,
    iters_count=101,
    loss_function=LogFunction(),
    iteration_callback=callback
)

Iteration 0
	Train perplexity:   2720.16685644
	Test perplexity:    2724.13392663
	Topics correlation  0.000952663363938
	Phi sparsity        0.0
	Theta sparsity      5.7431656329e-06
	Top5 PMI            0.00843759826024
	Top10 PMI           0.00964322052527
	Top20 PMI           0.0116327800629
	Avg kernel size     1962.1
	Avg kernel jacards  0.643754002782
Iteration 10
	Train perplexity:   2474.68233524
	Test perplexity:    2487.1992331
	Topics correlation  0.000945151795658
	Phi sparsity        0.0
	Theta sparsity      0.0475017229497
	Top5 PMI            0.0429180561786
	Top10 PMI           0.0329713654198
	Top20 PMI           0.0443627841964
	Avg kernel size     1888.83333333
	Avg kernel jacards  0.530887644845
Iteration 20
	Train perplexity:   1888.39191141
	Test perplexity:    1924.02433555
	Topics correlation  0.000904251384923
	Phi sparsity        0.0
	Theta sparsity      0.544968986906
	Top5 PMI            0.150593058918
	Top10 PMI           0.140182547574
	Top20 PMI         

In [24]:
D, W = train_n_dw_matrix.shape
T = 30

np.random.seed(42)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64) + 0.1
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = create_reg_lda(0.1, 0.)

_train_perplexity = artm_calc_perplexity_factory(train_n_dw_matrix) 
_test_perplexity = artm_calc_perplexity_factory(test_n_dw_matrix)
_top5_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 5)
_top10_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 10)
_top20_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 20)

def callback(it, phi, theta):
    if it % 10 == 0:
        print 'Iteration', it
        print '\tTrain perplexity:   {}'.format(_train_perplexity(phi, theta))
        print '\tTest perplexity:    {}'.format(_test_perplexity(phi, theta))
        print '\tTopics correlation  {}'.format(artm_calc_topic_correlation(phi))
        print '\tPhi sparsity        {}'.format(1. * np.sum(phi < 1e-20) / np.sum(phi >= 0))
        print '\tTheta sparsity      {}'.format(1. * np.sum(theta < 0.01) / np.sum(theta >= 0))
        print '\tTop5 PMI            {}'.format(_top5_pmi(phi))
        print '\tTop10 PMI           {}'.format(_top10_pmi(phi))
        print '\tTop20 PMI           {}'.format(_top20_pmi(phi))
        print '\tAvg kernel size     {}'.format(np.mean(artm_get_kernels_sizes(phi)))
        print '\tAvg kernel jacards  {}'.format(artm_get_avg_pairwise_kernels_jacards(phi))


phi, theta = artm_thetaless_em_optimization(
    n_dw_matrix=train_n_dw_matrix, 
    phi_matrix=phi_matrix,
    regularization_list=regularization_list,
    iters_count=101,
    iteration_callback=callback
)

Iteration 0
	Train perplexity:   2719.87370907
	Test perplexity:    2723.85738504
	Topics correlation  0.000950541115945
	Phi sparsity        3.19869725784e-05
	Theta sparsity      5.7431656329e-06
	Top5 PMI            0.00870857804955
	Top10 PMI           0.0097819585326
	Top20 PMI           0.011836723299
	Avg kernel size     1959.7
	Avg kernel jacards  0.640570548258
Iteration 10
	Train perplexity:   2319.43314924
	Test perplexity:    2329.93467986
	Topics correlation  0.000899696132646
	Phi sparsity        0.0387711186717
	Theta sparsity      0.0143694004135
	Top5 PMI            0.193866509528
	Top10 PMI           0.163233556828
	Top20 PMI           0.145329176171
	Avg kernel size     1606.56666667
	Avg kernel jacards  0.358299937967
Iteration 20
	Train perplexity:   2042.11350602
	Test perplexity:    2065.99922529
	Topics correlation  0.000516044198435
	Phi sparsity        0.0282706679461
	Theta sparsity      0.129697909488
	Top5 PMI            0.214488339015
	Top10 PMI           

In [25]:
D, W = train_n_dw_matrix.shape
T = 30

np.random.seed(42)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64) + 0.1
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = create_reg_lda(0., -0.1)

_train_perplexity = artm_calc_perplexity_factory(train_n_dw_matrix) 
_test_perplexity = artm_calc_perplexity_factory(test_n_dw_matrix)
_top5_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 5)
_top10_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 10)
_top20_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 20)

def callback(it, phi, theta):
    if it % 10 == 0:
        print 'Iteration', it
        print '\tTrain perplexity:   {}'.format(_train_perplexity(phi, theta))
        print '\tTest perplexity:    {}'.format(_test_perplexity(phi, theta))
        print '\tTopics correlation  {}'.format(artm_calc_topic_correlation(phi))
        print '\tPhi sparsity        {}'.format(1. * np.sum(phi < 1e-20) / np.sum(phi >= 0))
        print '\tTheta sparsity      {}'.format(1. * np.sum(theta < 0.01) / np.sum(theta >= 0))
        print '\tTop5 PMI            {}'.format(_top5_pmi(phi))
        print '\tTop10 PMI           {}'.format(_top10_pmi(phi))
        print '\tTop20 PMI           {}'.format(_top20_pmi(phi))
        print '\tAvg kernel size     {}'.format(np.mean(artm_get_kernels_sizes(phi)))
        print '\tAvg kernel jacards  {}'.format(artm_get_avg_pairwise_kernels_jacards(phi))


phi, theta = em_optimization(
    n_dw_matrix=train_n_dw_matrix, 
    phi_matrix=phi_matrix,
    theta_matrix=theta_matrix,
    regularization_list=regularization_list,
    iters_count=101,
    loss_function=LogFunction(),
    iteration_callback=callback
)

Iteration 0
	Train perplexity:   2720.04221623
	Test perplexity:    2724.12276922
	Topics correlation  0.000959412810628
	Phi sparsity        0.0
	Theta sparsity      5.7431656329e-06
	Top5 PMI            0.00843759826024
	Top10 PMI           0.00964322052527
	Top20 PMI           0.0116327800629
	Avg kernel size     1962.1
	Avg kernel jacards  0.643754002782
Iteration 10
	Train perplexity:   2468.1398764
	Test perplexity:    2481.24200012
	Topics correlation  0.000951899621516
	Phi sparsity        0.0
	Theta sparsity      0.0500344589938
	Top5 PMI            0.0429180561786
	Top10 PMI           0.0335836363043
	Top20 PMI           0.0443712010506
	Avg kernel size     1899.2
	Avg kernel jacards  0.521716475007
Iteration 20
	Train perplexity:   1886.37424435
	Test perplexity:    1923.57277845
	Topics correlation  0.000910463706454
	Phi sparsity        0.00920934019599
	Theta sparsity      0.544136227889
	Top5 PMI            0.15051137611
	Top10 PMI           0.140200309252
	Top20 PMI    

In [26]:
D, W = train_n_dw_matrix.shape
T = 30

np.random.seed(42)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64) + 0.1
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = create_reg_lda(0., -0.1)

_train_perplexity = artm_calc_perplexity_factory(train_n_dw_matrix) 
_test_perplexity = artm_calc_perplexity_factory(test_n_dw_matrix)
_top5_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 5)
_top10_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 10)
_top20_pmi = artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, train_n_dw_matrix.shape[0], 20)

def callback(it, phi, theta):
    if it % 10 == 0:
        print 'Iteration', it
        print '\tTrain perplexity:   {}'.format(_train_perplexity(phi, theta))
        print '\tTest perplexity:    {}'.format(_test_perplexity(phi, theta))
        print '\tTopics correlation  {}'.format(artm_calc_topic_correlation(phi))
        print '\tPhi sparsity        {}'.format(1. * np.sum(phi < 1e-20) / np.sum(phi >= 0))
        print '\tTheta sparsity      {}'.format(1. * np.sum(theta < 0.01) / np.sum(theta >= 0))
        print '\tTop5 PMI            {}'.format(_top5_pmi(phi))
        print '\tTop10 PMI           {}'.format(_top10_pmi(phi))
        print '\tTop20 PMI           {}'.format(_top20_pmi(phi))
        print '\tAvg kernel size     {}'.format(np.mean(artm_get_kernels_sizes(phi)))
        print '\tAvg kernel jacards  {}'.format(artm_get_avg_pairwise_kernels_jacards(phi))


phi, theta = artm_thetaless_em_optimization(
    n_dw_matrix=train_n_dw_matrix, 
    phi_matrix=phi_matrix,
    regularization_list=regularization_list,
    iters_count=101,
    iteration_callback=callback
)

Iteration 0
	Train perplexity:   2719.73958761
	Test perplexity:    2723.83701209
	Topics correlation  0.000957280449834
	Phi sparsity        3.19869725784e-05
	Theta sparsity      5.7431656329e-06
	Top5 PMI            0.00870857804955
	Top10 PMI           0.0097819585326
	Top20 PMI           0.011836723299
	Avg kernel size     1959.76666667
	Avg kernel jacards  0.640555200345
Iteration 10
	Train perplexity:   2310.9916771
	Test perplexity:    2321.9293276
	Topics correlation  0.000904327482639
	Phi sparsity        0.239658030184
	Theta sparsity      0.0145589248794
	Top5 PMI            0.196004379118
	Top10 PMI           0.160278850561
	Top20 PMI           0.14835066686
	Avg kernel size     1610.66666667
	Avg kernel jacards  0.349260840442
Iteration 20
	Train perplexity:   2038.41664836
	Test perplexity:    2063.10129431
	Topics correlation  0.000517554507265
	Phi sparsity        0.43036145279
	Theta sparsity      0.132328279348
	Top5 PMI            0.221438791557
	Top10 PMI          